# Pickling cuML Models for Persistence

This notebook demonstrates simple pickling of both single-GPU and multi-GPU cuML models for persistence

In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

## Single GPU Model Pickling

All single-GPU estimators are pickleable. The following example demonstrates the creation of a synthetic dataset, training, and pickling of the resulting model for storage. Trained single-GPU models can also be used to distribute the inference on a Dask cluster, which the `Distributed Model Pickling` section below demonstrates.

In [2]:
from cuml.datasets import make_blobs

X, y = make_blobs(n_samples=50,
                  n_features=10,
                  centers=5,
                  cluster_std=0.4,
                  random_state=0)

In [3]:
from cuml.cluster import KMeans

model = KMeans(n_clusters=5)

model.fit(X)

KMeans()

In [4]:
import pickle

pickle.dump(model, open("kmeans_model.pkl", "wb"))

In [5]:
model = pickle.load(open("kmeans_model.pkl", "rb"))

In [6]:
model.cluster_centers_

array([[-5.7684636 ,  2.3276033 , -3.7457774 , -1.8541752 , -5.1695833 ,
         7.6670885 ,  2.7118316 ,  8.495609  ,  1.7038486 ,  1.1884269 ],
       [ 4.647688  ,  8.37788   , -9.070581  ,  9.459332  ,  8.450422  ,
        -1.0210546 ,  3.392087  , -7.8629856 , -0.7527662 ,  0.48384127],
       [-2.9414442 ,  4.6401706 , -4.5027533 ,  2.2855108 ,  1.644645  ,
        -2.4937892 , -5.2241607 , -1.5499198 , -8.063638  ,  2.8169358 ],
       [-4.271077  ,  5.561165  , -5.6640916 , -1.8229512 , -9.2925    ,
         0.73028314,  4.4586773 , -2.8876226 , -5.1257744 ,  9.694357  ],
       [ 5.5837417 , -4.1515303 ,  4.369667  , -3.00205   ,  3.6388965 ,
        -4.341912  , -3.3187115 ,  6.503671  , -6.865036  , -1.0266498 ]],
      dtype=float32)

## Distributed Model Pickling

The distributed estimator wrappers inside of the `cuml.dask` are not intended to be pickled directly. The Dask cuML estimators provide a function `get_combined_model()`, which returns the trained single-GPU model for pickling. The combined model can be used for inference on a single-GPU, and the `ParallelPostFit` wrapper from the [Dask-ML](https://ml.dask.org/meta-estimators.html) library can be used to perform distributed inference on a Dask cluster.

In [7]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster

cluster = LocalCUDACluster()
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:35184 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 1 Memory: 251.80 GiB


In [8]:
from cuml.dask.datasets import make_blobs

n_workers = len(client.scheduler_info()["workers"].keys())

X, y = make_blobs(n_samples=5000, 
                  n_features=30,
                  centers=5, 
                  cluster_std=0.4, 
                  random_state=0,
                  n_parts=n_workers*5)

X = X.persist()
y = y.persist()

In [9]:
from cuml.dask.cluster import KMeans

dist_model = KMeans(n_clusters=5)

In [10]:
dist_model.fit(X)

In [11]:
import pickle

single_gpu_model = dist_model.get_combined_model()
pickle.dump(single_gpu_model, open("kmeans_model.pkl", "wb"))

In [12]:
single_gpu_model = pickle.load(open("kmeans_model.pkl", "rb"))

In [13]:
single_gpu_model.cluster_centers_

array([[ 4.8098726 ,  8.422669  , -9.239023  ,  9.379142  ,  8.499881  ,
        -1.0592815 ,  3.3437855 , -7.802613  , -0.59463334,  0.264476  ,
         5.5073934 , -4.1069803 ,  4.2890773 , -2.8172054 ,  3.6150153 ,
        -4.1612997 , -3.6209645 ,  6.2185307 , -6.9460483 , -1.0828305 ,
        -5.8267717 ,  2.2258766 , -3.8601213 , -1.6974075 , -5.3134165 ,
         7.5795794 ,  2.9187477 ,  8.540425  ,  1.5523206 ,  1.0841805 ],
       [-2.8941846 ,  4.4741874 , -4.4475665 ,  2.3820996 ,  1.7478833 ,
        -2.5046248 , -5.20833   , -1.6937683 , -8.134757  ,  2.6468303 ,
        -4.3163624 ,  5.565538  , -5.7322    , -1.7384952 , -9.344654  ,
         0.7084658 ,  4.43584   , -2.9008996 , -4.9486413 ,  9.695303  ,
         8.366522  , -6.2474546 , -6.3494725 ,  1.9546973 ,  4.1576157 ,
        -9.167906  ,  4.6070685 ,  8.788585  ,  6.8644214 ,  2.2319875 ],
       [-4.6657157 , -9.558956  ,  6.6572294 ,  4.440131  ,  2.1730306 ,
         2.5904028 ,  0.5800097 ,  6.2550354 , -8